<a href="https://colab.research.google.com/github/Ravi-shankar100/Fake-news-using-deep-learning/blob/master/FakeNewsClassifierUsingLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [147]:
import pandas as pd

In [148]:
df=pd.read_csv('train.csv')

In [149]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [150]:
###Drop Nan Values
df=df.dropna()


In [151]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [152]:
## Get the Dependent features
y=df['label']

In [153]:
X.shape

(18285, 4)

In [154]:
y.shape

(18285,)

In [155]:
import tensorflow as tf

In [156]:
tf.__version__

'2.2.0'

In [157]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [158]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [159]:
messages=X.copy()

In [160]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [161]:
messages.reset_index(inplace=True)

In [162]:
import nltk
import re
from nltk.corpus import stopwords

In [163]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [164]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
13443
1

In [165]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [166]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[4987, 1263, 3091, 4941, 1237, 2049, 1942, 640, 4805, 4532],
 [4219, 1568, 3257, 1130, 959, 2371, 3712],
 [1132, 110, 3989, 4156],
 [129, 2469, 3929, 2250, 4543, 2768],
 [4357, 959, 4940, 3509, 1335, 1761, 959, 4319, 3841, 618],
 [1483,
  2976,
  2616,
  624,
  666,
  3416,
  3123,
  831,
  475,
  669,
  3298,
  4593,
  4501,
  351,
  3712],
 [2317, 47, 1556, 2607, 4359, 2420, 2763, 3005, 4057, 1257, 882],
 [3021, 2560, 3308, 3225, 2114, 2083, 3416, 2024, 4057, 1257, 882],
 [2068, 3103, 407, 1289, 1825, 1871, 4427, 422, 3416, 1417],
 [451, 2271, 2027, 3353, 2450, 1425, 4806, 2749],
 [3077, 981, 3, 2756, 1751, 139, 4253, 3111, 91, 3406, 4373],
 [2250, 52, 1237, 1871, 3416, 2114],
 [2784, 4213, 4760, 4531, 1791, 3129, 2979, 4454, 3378],
 [1121, 3894, 541, 4180, 3450, 3229, 3912, 4057, 1257, 882],
 [1437, 3847, 3968, 1413, 2489, 4057, 1257, 882],
 [4127, 3002, 3734, 1459, 3321, 1882, 2375, 2193, 2682, 4487],
 [1814, 2579, 1568],
 [3523, 4080, 471, 1730, 3416, 3978, 3682, 3712],
 [3174, 1

### Embedding Representation

In [167]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  640 4805 4532]
 [   0    0    0 ...  959 2371 3712]
 [   0    0    0 ...  110 3989 4156]
 ...
 [   0    0    0 ... 4057 1257  882]
 [   0    0    0 ... 1969  888 2394]
 [   0    0    0 ... 3747 2823 2458]]


In [168]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4987,
       1263, 3091, 4941, 1237, 2049, 1942,  640, 4805, 4532], dtype=int32)

In [169]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 20, 40)            200000    
_________________________________________________________________
lstm_10 (LSTM)               (None, 100)               56400     
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [170]:
len(embedded_docs),y.shape

(18285, (18285,))

In [171]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [172]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [173]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [174]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 7s 37ms/step - loss: 0.3288 - accuracy: 0.8485 - val_loss: 0.1899 - val_accuracy: 0.9195
Epoch 2/10
192/192 [==============================] - 7s 35ms/step - loss: 0.1421 - accuracy: 0.9442 - val_loss: 0.1887 - val_accuracy: 0.9221
Epoch 3/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0998 - accuracy: 0.9629 - val_loss: 0.2086 - val_accuracy: 0.9195
Epoch 4/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0740 - accuracy: 0.9744 - val_loss: 0.2436 - val_accuracy: 0.9110
Epoch 5/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0544 - accuracy: 0.9821 - val_loss: 0.2737 - val_accuracy: 0.9148
Epoch 6/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0412 - accuracy: 0.9873 - val_loss: 0.3887 - val_accuracy: 0.9036
Epoch 7/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0315 - accuracy: 0.9904 - val_loss: 0.3628 - val_accuracy: 0.9165

### Adding Dropout 

In [175]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [176]:
y_pred=model.predict_classes(X_test)

In [177]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)